# 1. Programming Binomial Model

In [22]:
from model.binomial import EuropeanVanillaModel, AmericanModel
import pandas as pd

#for filling column headers
columns = lambda x: ['t = ' + str(i) for i in range(x + 1)]

### Vanilla European Call Example

In [23]:
european_call = EuropeanVanillaModel(20, 22, 3, 1, 1.2, .8, .08, 3, 'call')

##### Tree of stock prices</h5>

In [24]:
#s_tree.T will transpose the tree so it looks like how we build them by hand
st_df = pd.DataFrame(european_call.s_tree.T, columns=columns(european_call.n))
st_df

,t = 0,t = 1,t = 2,t = 3
0,20.0,24.0,28.8,34.56
1,NaN,16.0,19.2,23.04
2,NaN,NaN,12.8,15.36
3,NaN,NaN,NaN,10.24


##### Tree of derivative prices

In [25]:
#v_tree.T will transpose the tree so it looks like how we build them by hand
ot_df = pd.DataFrame(european_call.o_tree.T, columns=columns(european_call.n))
ot_df

,t = 0,t = 1,t = 2,t = 3
0,3.868797,5.734561,8.491440,12.56
1,NaN,0.444508,0.679918,1.04
2,NaN,NaN,0.000000,0.00
3,NaN,NaN,NaN,0.00


##### Optimal price at t=0</h5>

In [26]:
european_call.price()

3.8687965915200624

<h3>Vanilla European Put Example</h3>

In [27]:
european_put = EuropeanVanillaModel(80, 100, 2, 1, 1.2, .8, 0.04, 3, 'put')

##### Tree of stock prices

In [28]:
#s_tree.T will transpose the tree so it looks like how we build them by hand
st_df = pd.DataFrame(european_put.s_tree.T, columns=columns(european_put.n))
st_df

,t = 0,t = 1,t = 2,t = 3
0,80.0,96.0,115.2,138.24
1,NaN,64.0,76.8,92.16
2,NaN,NaN,51.2,61.44
3,NaN,NaN,NaN,40.96


##### Tree of derivative prices</h5>

In [29]:
#v_tree.T will transpose the tree so it looks like how we build them by hand
ot_df = pd.DataFrame(european_put.o_tree.T, columns=columns(european_put.n))
ot_df

,t = 0,t = 1,t = 2,t = 3
0,16.092361,9.105629,2.997768,0.00
1,NaN,28.311635,19.278944,7.84
2,NaN,NaN,44.878944,38.56
3,NaN,NaN,NaN,59.04


##### Optimal price at t=0</h5>

In [30]:
european_put.price()

16.092360501520012

<h3>American Put Example</h3>

In [31]:
american_put = AmericanModel(80, 100, 2, 1, 1.2, .8, 0.04, 3, option='put')

##### Tree of stock prices

In [32]:
#s_tree.T will transpose the tree so it looks like how we build them by hand
st_df = pd.DataFrame(american_put.s_tree.T, columns=columns(american_put.n))
st_df

,t = 0,t = 1,t = 2,t = 3
0,80.0,96.0,115.2,138.24
1,NaN,64.0,76.8,92.16
2,NaN,NaN,51.2,61.44
3,NaN,NaN,NaN,40.96


##### Tree of derivative prices</h5>

In [33]:
#v_tree.T will transpose the tree so it looks like how we build them by hand
ot_df = pd.DataFrame(american_put.o_tree.T, columns=columns(american_put.n))
ot_df

,t = 0,t = 1,t = 2,t = 3
0,20.0,10.604917,2.997768,0.00
1,NaN,36.000000,23.200000,7.84
2,NaN,NaN,48.800000,38.56
3,NaN,NaN,NaN,59.04


##### Optimal price at t=0</h5>

In [34]:
american_put.price()

20.0

In [35]:
american_call = AmericanModel(80, 100, 2, 1, 1.2, .8, 0.05, 3, 'call')
american_call.price()

8.1587070888752518